In [1]:
from IPython.display import display_html
import numpy as np
import cv2
from keras.models import model_from_json
from keras.preprocessing import image

json_file = open('count_fingers.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("count_fingers.h5")
print("Loaded model from disk")


numbers = ['FIVE', 'FOUR', 'NONE', 'ONE', 'THREE', 'TWO']

def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    

cap = cv2.VideoCapture(0)
while True:
    _,frame = cap.read()
    
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    cv2.rectangle(frame,(300,20),(100,300),(0,255,0),3)
 
    roi = frame_gray[20:300,100:300]  
    roi = cv2.resize(roi,(64,64))
    
    blur = cv2.GaussianBlur(roi, (7,7), 3)
    ad_thres = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    ret, thres = cv2.threshold(ad_thres, 25, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    image_array = image.img_to_array(thres)
    
    image_array = cv2.cvtColor(image_array,cv2.COLOR_GRAY2BGR)
    image_array = np.expand_dims(image_array,axis =0)
    
    predictions =loaded_model.predict(image_array)
    
    cv2.putText(frame,numbers[np.argmax(predictions)],(1,450), cv2.FONT_HERSHEY_SIMPLEX, 4,(255,255,255),2)
    cv2.imshow('Frame',frame)
    
    k =cv2.waitKey(15)
    if k ==27:
        break
cap.release()
cv2.destroyAllWindows()
restartkernel()

Loaded model from disk
1/1 [==============================] - 0s 28ms/step


KeyboardInterrupt: 